## 2. Construcción de la Red Neuronal

In [ ]:
# import packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import sklearn.metrics as metrics  ##matriz de confusion
from  sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#usando las libreiras tradicionales
from sklearn.metrics import precision_recall_fscore_support
#import xgboost as xgb
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import seaborn as sns
from sklearn import preprocessing

In [ ]:
# Ahora debemos importar las librerias de AX para poder realizar los experimentos!
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting # solo para graficar de otra manera!

In [ ]:
X_train=np.load('Datos/X_train.npy',allow_pickle='TRUE')
Y_train=np.load('Datos/Y_train.npy',allow_pickle='TRUE')

X_test=np.load('Datos/X_test.npy',allow_pickle='TRUE')
Y_test=np.load('Datos/Y_test.npy',allow_pickle='TRUE')

Y_train=to_categorical(Y_train)
Y_test=to_categorical(Y_test)

In [ ]:
# Esta función retorna un multi-layer-perceptron model en Keras.
# Utilizando algunos parametros que son los que van a ir variando.

def get_keras_model(num_hidden_layers, 
                    num_neurons_per_layer, 
                    dropout_rate, 
                    activation):
    
    # Creamos el MLP 
    
    # Definimos las capas
    ## Capa de entrada con dropout
    ## Esta es la forma más general de construir la red secuencial con KERAS.
    
    inputs = tf.keras.Input(shape=(X_train.shape[1],))  # input layer.
    x = layers.Dropout(dropout_rate)(inputs) # dropout on the weights.
    
    # Agregamos capas ocultas con dropout.
    for i in range(num_hidden_layers):
        x = layers.Dense(num_neurons_per_layer, 
                         activation=activation)(x)
        x = layers.Dropout(dropout_rate)(x)
    
    # Creamos la capa de salida
    # Tiene dos neuronas de salida porque mis datos target, son dos columnas.
    # Como es un problema de clasificación utilizo softmax que llevara los pesos a una probabilidad.
    # Lo que al predecir podremos llevar facilmente a una sola columna y luego calcular matriz de confusion.
    # Ojo: por si acaso, en la salida no hay dropout logicamente.
    
    outputs = layers.Dense(2, activation='softmax')(x)
    
    ##Finalmente, en esta linea construyo el modeo que esta secuencialmente conectado y tiene dropout
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model
    

    
# Esta funcion consume los hyperparametros and retorna un score (Cross validation).
def keras_mlp_cv_score(parameterization, weight=None):
    
    ## Aqui creo la red que probaré usando la función de arriba.
    model = get_keras_model(parameterization.get('num_hidden_layers'),
                            parameterization.get('neurons_per_layer'),
                            parameterization.get('dropout_rate'),
                            parameterization.get('activation'))
    
    ##imprimimos el modelo, para ver que esta funcionando y ver la cantidad de parametros que estamos utilizando.
    model.summary()
    
    ## Parametro en duro para poder configurar todas las redes con un largo fijo de EPOCAS
    ## Más elegante sería dejarlo como hiperparametro pero fijo en un default.
    NUM_EPOCHS = 50
    
    ############ 
    ## Acá configuramos el optimizador (adam, sgd, etc) y su learning rate.
    opt = parameterization.get('optimizer')
    opt = opt.lower()
    
    learning_rate = parameterization.get('learning_rate')
    
    if opt == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'rms':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    
    ###### LINEA CLAVE - SETEANDO LA COMPILACION
    # Ahora configuramos el entrenamiento con los parametros anteriores y que van a ir variando!
    # Estoy usando el error medio (no tiene mucho sentido en clasificación, pero para que pueda ver como se hace)
    # Más adelante corregiremos para una red orientada a clasificar.
    # En este caso lo que se quiere es MINIMIZAR el MSE! Si fuera accuracy, querriamos MAXIMIZAR
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['mse','accuracy'])
    
    ##Funcion utiliza la data como variable global.
    data = X_train
    #labels = y_train.values
    labels = Y_train
    
    # se entrena el modelo usando el 20% como validation set.
    res = model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=parameterization.get('batch_size'),
                    validation_split=0.2)
    
    # Finalmente, utilizamos las últimas de 10 epocas. Calculamos la media y desviación estandar del validation score.
    last10_scores = np.array(res.history['val_accuracy'][-10:])
    mean = last10_scores.mean()
    sem = last10_scores.std()
    
    # Si el modelo no converge seteamos una loss alta!
#    if np.isnan(mean):
#        return 9999.0, 0.0

    if np.isnan(mean):
        return 0.0, 0.0


    return mean, sem

In [ ]:
# Ahora la magia de AX
# Aqui configuramos los hiperparametros que deseamos probar en nuestros experimentos
# de manera bastante intuitiva.
# Importantes:
# "type" - puede ser un rango o puede ser choice.
# Si es "range:" - entonces, debo definir los limites del rango con "bounds:" [a..b] entre a y b
# y además debemos configurar de que manera se escogera el valor.. por ej. "log_scale:" es una escala logaritmica entre a y b
# o puede ser "value_type:" de tipo "int" o sea que es un rango de valores enteros entre a y b
#
# Si es "choice:" entonces, son valores específicos que quiero probar.
# Por lo tanto este parametro va de la mano con "values:" y ahi se le da la lista de valores.. [8, 16, 32, 64, 128, 256]
# o puede ser "values": ['tanh', 'sigmoid', 'relu'] ó "values": ['adam', 'rms', 'sgd']
#
# Como comentamos hay otras formas de hacer esto, sin embargo, requieren mucho más texto y es mucho menos claro e intuitivo
# que realizar las pruebas con AX.

parameters=[
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [0.0001, 0.5],
        "log_scale": True,
    },
    {
        "name": "dropout_rate",
        "type": "range",
        "bounds": [0.01, 0.5],
        "log_scale": True,
    },
    {
        "name": "num_hidden_layers",
        "type": "range",
        "bounds": [1, 3],
        "value_type": "int"
    },
    {
        "name": "neurons_per_layer",
        "type": "range",
        "bounds": [4, 16],
        "value_type": "int"
    },
    {
        "name": "batch_size",
        "type": "choice",
        "values": [8, 16, 32, 64, 128, 256],
    },
    
    {
        "name": "activation",
        "type": "choice",
        "values": ['tanh', 'sigmoid', 'relu'],
    },
    {
        "name": "optimizer",
        "type": "choice",
        "values": ['adam', 'rms', 'sgd'],
    },
]

In [ ]:
init_notebook_plotting()

ax_client = AxClient() #inicializo AX para los experimentos

# creo el experimento, donde en la variable parameters ingresan mis hiperparametros.
# luego en objective_name, configuro la métrica objetivo que deseo optimizar.
# finalmente debo decirle si quiero minimizar ó maximizar (True/False)
# MUY importante, en este caso, 'keras_cv' lo que indica es que optimizara la metrica que nosotros definimos
# en el compile de nuestra red! En este caso, es el MSE el error medio.
# Un buen MSE esta cerca de 0.. por eso, queremos minimizar 
ax_client.create_experiment(
    name="keras_experiment",
    parameters=parameters,
    objective_name='keras_cv',
    minimize=False)

def evaluate(parameters):
    return {"keras_cv": keras_mlp_cv_score(parameters)}

In [ ]:
## Aqui generamos 25 experimentos!! :-)
for i in range(4):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

In [ ]:
# look at all the trials.
ax_client.get_trials_data_frame().sort_values('trial_index')

In [ ]:
best_parameters, values = ax_client.get_best_parameters()

# the best set of parameters.
for k in best_parameters.items():
  print(k)

print()

# the best score achieved.
means, covariances = values
print(means)

# Evaluando los mejores parametros

In [ ]:
# train the model on the full training set and test.
keras_model = get_keras_model(best_parameters['num_hidden_layers'], 
                              best_parameters['neurons_per_layer'], 
                              best_parameters['dropout_rate'],
                              best_parameters['activation'])

opt = best_parameters['optimizer']
opt = opt.lower()

learning_rate = best_parameters['learning_rate']

if opt == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
elif opt == 'rms':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

NUM_EPOCHS = 100

# Specify the training configuration.
keras_model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mse','accuracy'])

data = X_train
#labels = y_train.values
labels = Y_train

hist = keras_model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=best_parameters['batch_size'], validation_split = 0.2)

In [ ]:
keras_model.summary()